# DineSafe Toronto: Feature Engineering

This notebook creates new columns or aggregations that'll help answer deeper questions:
- Which establishments are higher risk? (risk scores)
- Do some establishments repeatedly violate safety codes? 
- Which establishments have crucial infractions? (binary flags)
- How do violations vary over time, geography, and establishment type?

## Load the latest raw DineSafe CSV Data

In [96]:
import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"

csv_files = list(PROCESSED_DIR.glob("dinesafe_*.csv"))

if not csv_files:
    raise FileNotFoundError(f"No processed DineSave CSV files found in {[PROCESSED_DIR.resolve()]}")

latest_file = max(csv_files, key=lambda f: f.stat().st_mtime)

print(f"Loading {latest_file.name}")
df = pd.read_csv(latest_file)

Loading dinesafe_feature_for_excel.csv


In [97]:
# Convert types
df['Inspection Date'] = pd.to_datetime(df['Inspection Date'])
df['Inspection ID'] = df['Inspection ID'].astype('Int64')

## Create Time-Based Features

In [98]:
df["Inspection Month"] = pd.to_datetime(df["Inspection Date"]).dt.month

In [99]:
df["Inspection Year"] = pd.to_datetime(df["Inspection Date"]).dt.year

In [100]:
df["Inspection Period"] = df["Inspection Date"].values.astype("datetime64[M]") # month start

In [101]:
df[['Inspection Date', 'Inspection Month', 'Inspection Year', 'Inspection Period']].head(5)

,Inspection Date,Inspection Month,Inspection Year,Inspection Period
0,2023-03-07,3.0,2023.0,2023-03-01
1,2023-03-07,3.0,2023.0,2023-03-01
2,2023-08-25,8.0,2023.0,2023-08-01
3,2022-08-10,8.0,2022.0,2022-08-01
4,2022-08-10,8.0,2022.0,2022-08-01


## Create Flagging Features

In [102]:
df["has_infraction"] = df["Infraction Details"].notna()

In [103]:
df["is_crucial"] = df["Severity"].eq("C - Crucial")

In [104]:
df["is_serious"] = df["Severity"].isin(["C - Crucial", "S - Significant"])

In [105]:
df["Severity"].unique()

array(['M - Minor', nan, 'C - Crucial', 'S - Significant',
       'NA - Not Applicable'], dtype=object)

In [106]:
df["has_action_taken"] = df["Action"].notna()

In [107]:
df["was_fined"] = df["Amount Fined"] > 0

In [108]:
df[["Infraction Details", "has_infraction", "Severity", "is_crucial", "is_serious", "Action"]].head(10)

,Infraction Details,has_infraction,Severity,is_crucial,is_serious,Action
0,FOOD PREMISE NOT MAINTAINED WITH CLEAN FLOORS ...,True,M - Minor,False,False,Notice to Comply
1,Operate food premise - equipment not arranged ...,True,M - Minor,False,False,Notice to Comply
2,NaN,False,NaN,False,False,NaN
3,Fail to protect against entry of pests - Sec. ...,True,M - Minor,False,False,Notice to Comply
4,Store potentially hazardous foods at internal ...,True,C - Crucial,True,True,Notice to Comply
5,FOOD PREMISE NOT MAINTAINED WITH CLEAN FLOORS ...,True,M - Minor,False,False,Notice to Comply
6,Fail to retain records of pest control measure...,True,S - Significant,False,True,Notice to Comply
7,Fail to protect against harbouring of pests - ...,True,S - Significant,False,True,Notice to Comply
8,NaN,False,NaN,False,False,NaN
9,Food premise maintained in manner to permit co...,True,C - Crucial,True,True,Notice to Comply


In [109]:
null_rows = df[df['Amount Fined'].isna()].head(2)
notnull_rows = df[df['Amount Fined'].notna()].head(2)
combined = pd.concat([null_rows, notnull_rows])
combined[['Amount Fined', 'was_fined']]

,Amount Fined,was_fined
0,NaN,False
1,NaN,False
1041,875.0,True
1636,1567.5,True


In [110]:
df["total_inspections"] = df.groupby("Establishment ID")["Inspection ID"].transform("count")

In [111]:
df["infraction_count"] = df.groupby("Establishment ID")["has_infraction"].transform("sum")

In [112]:
df["infraction_rate"] = df["infraction_count"] / df["total_inspections"]

In [113]:
# Group by 'Establishment ID' and show only the specified columns
df.groupby("Establishment ID")[["total_inspections", "infraction_count", "infraction_rate"]].head(5)

,total_inspections,infraction_count,infraction_rate
0,7,5,0.714286
1,7,5,0.714286
2,7,5,0.714286
3,37,36,0.972973
4,37,36,0.972973
...,...,...,...
136714,2,1,0.5
136718,2,1,0.5
136722,1,1,1.0
136723,5,0,0.0


## Export Data to Excel

In [114]:
pd.DataFrame({'Column': df.columns, 'Type': df.dtypes.values})

,Column,Type
0,Establishment ID,int64
1,Inspection ID,Int64
2,Establishment Name,object
3,Establishment Type,object
4,Establishment Address,object
5,Establishment Status,object
6,Infraction Details,object
7,Inspection Date,datetime64[ns]
8,Inspection Period,datetime64[s]
9,Severity,object


In [116]:
cols_for_excel = [
    "Establishment ID",
    "Inspection ID",
    "Establishment Name", 
    "Establishment Type",
    "Establishment Address",
    "Establishment Status",
    "Infraction Details",
    "Inspection Date",
    "Inspection Period",
    "Severity",
    "Action",
    "Outcome",
    "Amount Fined",
    "Latitude",
    "Longitude",
    "Inspection Month",
    "Inspection Year",
    "has_infraction",
    "is_crucial",
    "is_serious",
    "has_action_taken",
    "was_fined",
    "total_inspections",
    "infraction_count",
    "infraction_rate"
]

df_excel = df[cols_for_excel].copy()
out_csv = PROCESSED_DIR / "dinesafe_feature_for_excel.csv"
df_excel.to_csv(out_csv, index=False)
print(f"Saved: {out_csv}")

Saved: /Users/chrisdeng/Projects/personal/dinesafe-toronto/data/processed/dinesafe_feature_for_excel.csv
